## 웹 크롤링 

### 인터넷 접속 라이브러리 추가

- urllib.request 모듈로 작업
- 일반 html 소스 로드로 별다른 문제 없음

In [8]:
from urllib.request import urlopen, Request

# 도시별 날씨 검색함수
def get_weather(city):
    # 기상청 홈페이지 도시별 날씨 페이지
    url = 'https://www.weather.go.kr/w/obs-climate/land/city-obs.do'
    page = urlopen(url=url)
    # 인코딩은 utf-8로 되어있다.
    text = page.read().decode('utf-8')
    # 텍스트를 찾는데 꺽쇠 뒤에 city를 찾아라
    text = text[text.find(f'>{city}</a'):]

    # 기온 가져오기
    for i in range(7):
        # 반복하면서 <tb>에 둘러쌓인 원하는 값 찾기
        text = text[text.find(f'<td>')+1:]
    # 찾은 지점부터 3번 째 값
    start = 3
    # 사이값 찾은 뒤에 있는 </tb>값
    end = text.find('</td>')
    # 시작지점 ~ 종료지점 사이 값
    current_temp = text[start:end]
    print(f'{city}의 현재 기온은 {current_temp}℃ 입니다.')

    # 습도 가져오기
    for i in range(3):
        text = text[text.find(f'<td>')+1:]
    start = 3
    end = text.find('</td>')
    current_humid = text[start:end]
    print(f'{city}의 현재 습도는 {current_humid}% 입니다.')
    
if __name__ == '__main__':
    get_weather('부산')

부산의 현재 기온은 7.4℃ 입니다.
부산의 현재 습도는 41% 입니다.


### OpenAPI 크롤링

- 공공데이터 포털
    - https://www.data.go.kr
    - 부산광역시 버스정보안내기 현황 

```PYTHON    
    # SSL 문제 때문에 계속 에러나서 진행이 안됐음
    response = requests.get(total_url, verify=False)
```
- 문제점
    - https를 일반 request로 부르면 SSL 오류발생
    - 해결하려면 외부모듈 request를 사용, (검증)verify=False 옵션을 지정
    - 아래 셀의 39번 라인

In [9]:
import requests # 기본적인 URL 모듈로는 안되서 대체
import json
import ssl
from urllib.parse import quote, unquote, urlencode  # 한글을 URLencode 변환하는 함수

def getRequestUrl(url):
    req = Request(url)

    try:
        res = urlopen(req)
        # 응답이 200으로 나오면 정상적으로 통신된다는 이야기
        if res.getcode() == 200:
            return res.read().decode('utf-8')
    except Exception as e:
        print(e)
        return None

# stationName 정류소 이름
def getDataPortalSearch(stationName, type):
    
    # 고정값인 페이지 주소 + 특수 키
    api_url = 'https://apis.data.go.kr/6260000/BusanTblBusinfoeqStusService/getTblBusinfoeqStusInfo'
    # 여까지는 일반적인 URL
    queryString = "?" + urlencode(
        {   
            # url을 인코딩해서 특수문자 변환해줌
            'serviceKey' : '6hhxOoRZmduvmq1x2rC8tUpOTEJPythkOXqaCfRhb1G8rL++dNSwoN9DEGcZKHGhumwHaWyhtgGXbNDBbE/J9g==',
            # 페이지 넘버
            '&pageNo' : '1',
            '&numOfRows' : '10',
            # 롯데백화점 이름 자동으로 인코딩
            'stationLoc' : stationName,
            'resultType' : type
            # 시작은 ? 뒤에 임마들은 파라미터(서비스키, &는 구분자, 페이지 넘버 등등)
        }
    )
    total_url = api_url + queryString
    # SSL 문제 때문에 계속 에러나서 진행이 안됐음
    response = requests.get(total_url, verify=False)
    return response.text
    
try:
    result = getDataPortalSearch('백화점', 'json')
    # json 읽기
    json_data = json.loads(result)
    # 딕셔너리 형태로 리스트 생성
    station_data = json_data['getTblBusinfoeqStusInfo']['body']['items']['item']
    # 리스트 순번대로 출력
    for item in station_data:
        print(item)
except Exception as e:
    print('찾는 데이터가 없습니다.')

{'stationNum': '09283', 'stationLoc': '롯데백화점센텀시티점', 'lat': '35.16967324', 'lng': '129.1317843', 'addr': '해운대구 우동 1496', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '09336', 'stationLoc': '롯데백화점센텀시티점', 'lat': '35.16997953', 'lng': '129.1320357', 'addr': '해운대구 우동 1498', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '05718', 'stationLoc': '서면역.롯데호텔백화점', 'lat': '35.15776525', 'lng': '129.0566948', 'addr': '부산진구 부전동 576-1', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '05719', 'stationLoc': '서면역.롯데호텔백화점', 'lat': '35.15748116', 'lng': '129.0561774', 'addr': '부산진구 부전동 576-1', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '06030', 'stationLoc': '롯데백화점동래점.명륜역', 'lat': '35.21276314', 'lng': '129.0781263', 'addr': '동래구 온천1동 474-25', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '06046', 'stationLoc': '롯데백화점동래점', 'lat': '35.2101971', 'lng': '129.0778307', 'addr': '동래구 온천2동 503-46', 'insYear': '', 'dataDay': '2023-02-03'}


e:\DEV\Langs\python 3.11\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


### OpenAPI로 가져온 데이터 지도 표시

```python
stop_str = '<h4>' + item['stationLoc'] + '</h4>' + item['stationNum'] + '<br>' + item['addr']
iframe = folium.IFrame(stop_str)
```

- 문제점
    - 딕셔너리 구조하고 문자열 포맷팅 f'{}' 호환 안된다.
    - 구식방법인 문자열 결합 방식으로 해결 

In [10]:
import folium

if len(station_data) > 0:       
    # 정류소 중 제일 첫번째 인덱스 정류소 위경도를 중심으로
    center_lat = station_data[0]['lat']
    center_lng = station_data[0]['lng']
    
m = folium.Map(location=[center_lat, center_lng],
               zoom_start=13)

# 전체 정류소 위치값 마커 표시
for item in station_data:
        # stop_str = '<h4>' + item['stationLoc'] + '</h4>' + item['stationNum'] + '<br>' + item['addr']
        stop_str = f'<h4>{item["stationLoc"]}</h4>{item["stationNum"]}<br>{item["addr"]}'
        iframe = folium.IFrame(stop_str)
        folium.Marker(location=[item['lat'],
                                item['lng']],
                                popup=folium.Popup(iframe,
                                                   min_width=200, 
                                                   max_width=250),
                                icon=folium.Icon(color='blue', 
                                                 icon='pushpin')
                      ).add_to(m)
m

In [11]:
import folium

if len(station_data) > 0:       
    # 정류소 중 제일 첫번째 인덱스 정류소 위경도를 중심으로
    center_lat = station_data[0]['lat']
    center_lng = station_data[0]['lng']
    
m = folium.Map(location=[center_lat, center_lng],
               zoom_start=13)

# 전체 정류소 위치값 마커 표시
for item in station_data:
        folium.Marker(location=[item['lat'],
                                item['lng']],
                                popup=folium.Popup(folium.IFrame('<h4>' + item['stationLoc'] + 
                                                                 '</h4>' + item['stationNum'] + 
                                                                 '<br>' + item['addr']),
                                                   min_width=200, 
                                                   max_width=250),
                                icon=folium.Icon(color='blue', 
                                                 icon='pushpin')
                     ).add_to(m)
m

### BeautifulSoup (version 4)

웹크롤링을 편하게 해주는 도구(라이브러리)

```shell
pip install beautifulsoup4
```

In [12]:
!pip install beautifulsoup4

'pip'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


#### 모듈 import

```python
from bs4 import BeaurifulSoup
```

In [13]:
from bs4 import BeautifulSoup
import requests

url = 'https://kin.naver.com/search/list.naver?query=%EC%A0%9C%EC%9E%84%EC%8A%A4+%EC%9B%B9+%EB%A7%9D%EC%9B%90%EA%B2%BD'

response = requests.get(url)

if response.status_code == 200: # webpage OK
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')   # 파싱 - 분석해서 이용자가 보기좋게 만드는 것
    # print(soup)
    title = soup.select_one('#s_content > div.section > ul > li:nth-child(1) > dl > dt > a')
    # 글자만 추출해서 가지고 온다.
    print(title.get_text())
else:
    print(f'Error : {response.status_code}')

제임스웹 우주망원경 질문이요


In [14]:
from bs4 import BeautifulSoup
import requests

url = 'https://kin.naver.com/search/list.naver?query=%EC%A0%9C%EC%9E%84%EC%8A%A4+%EC%9B%B9+%EB%A7%9D%EC%9B%90%EA%B2%BD'

response = requests.get(url)

if response.status_code == 200: # webpage OK
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')   # 파싱 - 분석해서 이용자가 보기좋게 만드는 것
    # 위치 지정
    ul = soup.select_one('ul.basic1')
    titles = ul.select('li > dl > dt > a')
    # 타이틀 수 만큼 반복하면서 텍스트만 가지고 온다.
    for title in titles:
        print(title.get_text())
else:
    print(f'Error : {response.status_code}')

제임스웹 우주망원경 질문이요
최고고도 부탁드립니다. 제임스웹 망원경이
제임스 웹 망원경 우주사진 원본 어디서... 
제임스 웹 망원경
제임스 웹 망원경 촬영 원리
제임스 웹 우주망원경이 찍은 첫 사진은... 
제임스 웹 망원경의 종류 알려주세요
제임스 웹 우주망원경
제임스웹 망원경
제임스 웹 우주 망원경 제작 기간이
